## Student details
Student name: **Siddharth Prince**  
Student id: **23052058**

# Task 1

## A. Downloading Peter Norwig's unigram corpus

In [6]:
# Housekeeping - imports and utility functions (taken from my previous code for Etivity 3)
import pandas as pd
import jellyfish

def processNGramData(ngramContent):
    data = {'Word': [], 'Count': []}
    for word in ngramContent:
        dataPoint = word.split('\t')
        data['Word'].append(dataPoint[0])
        data['Count'].append(int(dataPoint[1]))
    return pd.DataFrame(data)

# Method to search the bigram data for the seed word and get all corresponding matches.
def searchDataFrame(df, searchTerm: str, columnToSearch: str, matchWord: bool=True):
    searchRegex = f"^{searchTerm}$" if matchWord else searchTerm
    return df.loc[df[columnToSearch].str.contains(searchRegex, regex=True)]

In [3]:
# Downloading unigram data
!wget https://norvig.com/ngrams/count_1w.txt
with open("./count_1w.txt", "r") as unigramFile:
    unigramContent = unigramFile.read().splitlines()
unigrams_df = processNGramData(unigramContent)
totalUnigrams = len(unigrams_df)
display(unigrams_df.head(100))

--2023-11-03 13:16:49--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt’

count_1w.txt        100%[===================>]   4.73M   733KB/s    in 6.8s    

2023-11-03 13:16:57 (711 KB/s) - ‘count_1w.txt’ saved [4956241/4956241]



,Word,Count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
95,like,520585287
96,service,519537222
97,x,508609523
98,than,502609275


## B. Spelling correction function that prints out possible candidates to replace the misspelt word

In [5]:
# Computing all the probabilities for each unigram and adding a column to the unigram data frame
totalUnigramCount = unigrams_df['Count'].sum()
print(f'Total unigram count: {totalUnigramCount}')
unigrams_df['P(Word)'] = unigrams_df['Count']/totalUnigramCount
display(unigrams_df.head(100))

Total unigram count: 588124220187


,Word,Count,P(Word)
0,the,23135851162,0.039338
1,of,13151942776,0.022363
2,and,12997637966,0.022100
3,to,12136980858,0.020637
4,a,9081174698,0.015441
...,...,...,...
95,like,520585287,0.000885
96,service,519537222,0.000883
97,x,508609523,0.000865
98,than,502609275,0.000855


In [33]:
def nonWordSpellingCorrection(nonWord):
    splits = [(nonWord[:i], nonWord[i:]) for i in range(len(nonWord) + 1)]
    candidates = pd.DataFrame()
    for split in splits:
        left, right = split
        left_candidates = searchDataFrame(unigrams_df, left, 'Word', False)
        right_candidates = searchDataFrame(unigrams_df, right, 'Word', False)
        # display(left_candidates)
        potential_candidates = pd.concat([left_candidates, right_candidates])
        # display(potential_candidates)
        # for row in potential_candidates:
        #     wordToCheck = row['Word'].values[0]
        #     print(wordToCheck) #unable to find a way in time to itereate over the df and get the values alone to pass into the function. :(
        filtered_candidates = potential_candidates.loc[lambda x: jellyfish.damerau_levenshtein_distance(nonWord, x['Word'].values) <= 2]
        candidates = pd.concat([candidates, filtered_candidates])
    print(f'nonWord = {nonWord}')
    display(candidates)
    return candidates

In [34]:
candidates_df = nonWordSpellingCorrection('acress')

TypeError: argument 'b': 'ndarray' object cannot be converted to 'PyString'

# Task 2

In [23]:
from textblob import Word

def getSpellingSuggestions(word):
    return Word(word).spellcheck()

print(f'String={"acress"}')
print(f'candidates = {getSpellingSuggestions("acress")}')

String=acress
candidates = [('across', 0.6851851851851852), ('access', 0.1728395061728395), ('acres', 0.1111111111111111), ('actress', 0.021604938271604937), ('caress', 0.009259259259259259)]


# Task 3

In [24]:
from textblob import TextBlob

def correctSentence(sentence):
    return TextBlob(sentence).correct()

print(f'sentence={"I have good speling"}\nCorrected sentence= {correctSentence("I have good speling")}')

sentence=I have good speling
Corrected sentence= I have good spelling
